In [8]:
import cufflinks as cf
from plotly.offline import iplot
import pandas as pd
import numpy as np

#modo offiline plotly
cf.go_offline()

### Cria um dataframe aleatorio

In [18]:
df = pd.DataFrame(np.random.randn(100, 5), columns=['A','B','C','D', 'E'])

In [19]:
df.head()

,A,B,C,D,E
0,-0.936554,0.074087,-0.458205,0.721157,-0.227474
1,1.240904,-0.841002,-1.400342,-0.016603,2.163676
2,-0.494489,0.696811,0.957306,-0.950051,-1.159281
3,-0.203022,0.406513,-0.173564,0.825877,0.841834
4,-1.515315,-0.467948,0.767918,-0.303841,0.511533


In [20]:
df.iplot()

In [27]:
df = pd.read_csv('olist_classified_public_dataset.csv')

In [28]:
df.head()

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,...,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
0,0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01.000000,2017-08-30 11:55:08.970352,2017-09-21 00:00:00.000000,...,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido
1,1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36.000000,2017-09-26 09:28:10.922048,2017-10-24 00:00:00.000000,...,0,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido
2,2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42.000000,2018-01-17 07:29:56.419769,2018-02-05 00:00:00.000000,...,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega
3,3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42.000000,2018-02-06 05:31:50.990164,2018-03-13 00:00:00.000000,...,3,0,0,0,0,0,0,0,atrasado,problemas_de_entrega
4,4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42.000000,2017-12-08 02:36:49.587515,2018-01-03 00:00:00.000000,...,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega


## Perguntas 
- Quais a distribuicao dos status dos pedidos?
- Quais os meses do ano houve mais vendas?
- Qual a quantidade de items de um pedido? (medio)
- Qual a quantidade de vendedores em um pedido ? (medio)
- O valor do frete tende aumentar com o preco do produto?
- Qual o tempo medio de entrega?
- Como foi as vendas por mês?
- Quais meses superaram a meta de vendas?
- Como foi a venda por mês se comparado ao ano de 2016?

In [26]:
df.order_status.value_counts().iplot(kind='bar', title='Status dos pedidos')

### Transformando em datetime

In [29]:
df.order_purchase_timestamp = pd.to_datetime(df.order_purchase_timestamp)

### Extraindo o mês das datas

In [30]:
df['order_purchase_month'] = df.order_purchase_timestamp.dt.to_period('M').astype(str)

**Cria a variável `vendas_por_mes` que é a soma do valor dos produtos agrupados por mês**

In [32]:
vendas_por_mes = df.groupby(by='order_purchase_month').order_products_value.sum()

# Agora será direto com plotly

In [42]:
import plotly.offline as py
import plotly.graph_objs as go

# habilita o modo offline
plotly.offline.init_notebook_mode(connected=True)

- **por padrão o iplot() plota um gráfico de linha, da sentido de valor acumulado**

In [43]:
data = [go.Scatter(x=vendas_por_mes.index,
                   y=vendas_por_mes.values)]

py.iplot(data)

## Grafico de barras

- **Customizando graficos de barras: titulos e labels dos eixos**

In [44]:
data = [go.Bar(x=vendas_por_mes.index,
               y=vendas_por_mes.values,
               marker = {'color': 'lightblue'})]

# Criando Layout
configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                 yaxis={'title':'Valores em Vendas'},
                                 xaxis={'title': 'Periodo'})

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico
py.iplot(fig)

- **Parâmetro opacity e width**

In [45]:
data = [go.Bar(x=vendas_por_mes.index,
               y=vendas_por_mes.values,
               marker = {'color': 'lightblue',
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               opacity= 0.7
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                 yaxis={'title':'Valores em Vendas'},
                                 xaxis={'title':'Periodo'})

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico
py.iplot(fig)

## Alterando as cores para valores menores que a media de vendas mensais

In [46]:
cores = []

In [47]:
media = vendas_por_mes.values.mean()

In [48]:
# se o valor de vendas do mês for menor que a média a lista de cor será vermelha, senão azul.
for x in vendas_por_mes.values:
    if x < media:
        cores.append('red')
    else:
        cores.append('lightblue')

In [55]:
data = [go.Bar(x=vendas_por_mes.index,
               y=vendas_por_mes.values,
               marker = {'color': cores,  #lista de cores
                         'line': {'color': '#333',
                                  'width': 2}
                        },
               opacity= 0.7
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                 yaxis={'title':'Valores em Vendas'},
                                 xaxis={'title':'Periodo'})

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico
py.iplot(fig)

**Destaca a barra com o maior valor e minimiza a visao de todas as outras**

In [56]:
maximo_de_vendas = vendas_por_mes.values.max()

In [57]:
cores = []
for x,y in zip(vendas_por_mes.values, vendas_por_mes.index):
    if x == maximo_de_vendas:
        mes_maximo_de_vendas = y
        cores.append('blue')
    else:
        cores.append('lightgray')

**Plota o gráfico de vendas com um destaque de cor e anotação para o mês que teve mais vendas**

In [58]:
data = [go.Bar(x=vendas_por_mes.index,
               y=vendas_por_mes.values,
               marker = {'color': cores,
                         'line': {'color': '#333',
                                 'width': 2}
                        },
               opacity= 0.7
              )
       ]

# Criando Layout
configuracoes_layout = go.Layout(title='Vendas no Periodo',
                                 yaxis={'title':'Valores em Vendas'},
                                 xaxis={'title': 'Periodo'},
                                 # texto na barra de destaque
                                 annotations = [{'text':'Mês destaque de vendas',
                                                'x':mes_maximo_de_vendas,
                                                'y':maximo_de_vendas}
                                               ]
                                                                                              
                                )

# Objeto figura

fig = go.Figure(data=data, layout=configuracoes_layout)

# plotando o grafico
py.iplot(fig)